<a href="https://colab.research.google.com/github/ordevoir/Python/blob/main/15.8_%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D1%8B_-_Property.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Свойство** (`property`) – это встроенный механизм Python, позволяющий определить методы доступа к атрибуту (геттер, сеттер, делитер) и использовать их через синтаксис обычного атрибута. Под капотом `property` реализован как дескриптор – объект, определяющий методы `__get__()`, `__set__()` и/или `__delete__()`. Поскольку `property` – дескриптор данных (имеет `__set__`), он имеет наивысший приоритет и перехватывает доступ к атрибуту.

Убедимся, что встроенный класс `property` реализует протокол дескриптора:

In [1]:
print(f"property имеет __get__: {hasattr(property, '__get__')}")
print(f"property имеет __set__: {hasattr(property, '__set__')}")
print(f"property имеет __delete__: {hasattr(property, '__delete__')}")

property имеет __get__: True
property имеет __set__: True
property имеет __delete__: True


## Базовый пример использования

Сигнатура конструктора `property`:

```python
property(fget=None, fset=None, fdel=None, doc=None)
```

где `fget`, `fset`, `fdel` – функции для чтения, записи и удаления атрибута соответственно.

Рассмотрим класс `Cell`, в котором размер хранится как защищённый атрибут (`_size`), а доступ к нему осуществляется через свойство `size` с валидацией:

In [2]:
class Cell:
    """Биологическая клетка с контролируемым размером."""

    def __init__(self, size):
        self.size = size  # использует сеттер
        self.is_alive = True

    # --- Геттер ---
    def _get_size(self):
        print("  [getter] Чтение размера")
        return self._size

    # --- Сеттер ---
    def _set_size(self, value):
        print(f"  [setter] Установка размера = {value}")
        if value <= 0:
            raise ValueError("Размер клетки должен быть положительным")
        if value > 1000:
            raise ValueError("Размер клетки не может превышать 1000 мкм")
        self._size = value

    # --- Делитер ---
    def _del_size(self):
        print("  [deleter] Удаление размера")
        del self._size

    # Создание property через конструктор
    size = property(_get_size, _set_size, _del_size, "Размер клетки в микрометрах")

In [3]:
cell = Cell(50)
print(f"Размер: {cell.size} мкм")      # чтение

cell.size = 75                          # запись
print(f"Новый размер: {cell.size} мкм")

  [setter] Установка размера = 50
  [getter] Чтение размера
Размер: 50 мкм
  [setter] Установка размера = 75
  [getter] Чтение размера
Новый размер: 75 мкм


In [ ]:
# cell.size = -10  # валидация в сеттере

## Декораторный синтаксис

Более распространённый и читаемый способ – использование `property` как декоратора:

In [4]:
import math

class Cell:
    """Биологическая клетка (декораторный синтаксис)."""

    def __init__(self, size):
        self.size = size
        self.is_alive = True

    @property
    def size(self):
        """Радиус клетки в микрометрах."""
        return self._size

    @size.setter
    def size(self, value):
        if value <= 0:
            raise ValueError("Радиус должен быть положительным")
        self._size = value

    @size.deleter
    def size(self):
        del self._size

    # Вычисляемое свойство (только для чтения)
    @property
    def volume(self):
        """Объём клетки (приближение к сфере), только чтение."""
        return (4/3) * math.pi * self._size ** 3

    @property
    def surface_area(self):
        """Площадь поверхности клетки."""
        return 4 * math.pi * self._size ** 2

In [ ]:
cell = Cell(10)
print(f"Радиус: {cell.size} мкм")
print(f"Объём: {cell.volume:.2f} мкм³")
print(f"Площадь поверхности: {cell.surface_area:.2f} мкм²")

Радиус: 10 мкм
Объём: 4188.79 мкм³
Площадь поверхности: 1256.64 мкм²


In [5]:
# cell.volume = 5000

## Интроспекция

Property хранится как атрибут класса, а у объекта property есть атрибуты `fget`, `fset`, `fdel`:

In [6]:
Cell.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'Биологическая клетка (декораторный синтаксис).',
              '__init__': <function __main__.Cell.__init__(self, size)>,
              'size': <property at 0x7d1ec9dcd5d0>,
              'volume': <property at 0x7d1ec9db10d0>,
              'surface_area': <property at 0x7d1ec9dcda80>,
              '__dict__': <attribute '__dict__' of 'Cell' objects>,
              '__weakref__': <attribute '__weakref__' of 'Cell' objects>})

In [7]:
prop = Cell.__dict__['size']
print(f"Cell.size — это property: {prop}")

Cell.size — это property: <property object at 0x7d1ec9dcd5d0>


In [8]:
print(f"\nАтрибуты объекта property:")
print(f"  fget: {prop.fget}")
print(f"  fset: {prop.fset}")
print(f"  fdel: {prop.fdel}")


Атрибуты объекта property:
  fget: <function Cell.size at 0x7d1ec9e093a0>
  fset: <function Cell.size at 0x7d1ec9e09440>
  fdel: <function Cell.size at 0x7d1ec9e094e0>


### Ручной вызов методов дескриптора

Мы можем вызвать методы `__get__`, `__set__`, `__delete__` напрямую – результат будет таким же, как при обычном доступе к атрибуту.

`c.size` эквивалентно `prop.__get__(c, Cell)`:

In [9]:
c = Cell(25)
prop = Cell.__dict__['size']

print(f"Через атрибут: {c.size}")
print(f"Через __get__: {prop.__get__(c, Cell)}")

Через атрибут: 25
Через __get__: 25


In [10]:
prop.__set__(c, 7)
c.size

7

## Переопределение декоратора

Пусть мы хотим переопределить сеттер в подклассе. Для этого можно воспользоваться следующим синтаксисом:

In [11]:
# Вариант 1: прямая запись в защищённый атрибут
class LargeCell(Cell):
    @Cell.size.setter
    def size(self, value):
        if value > 500:
            raise ValueError("Слишком большой радиус")
        self._size = value

In [12]:
# Вариант 2: вызов родительского сеттера
class LargeCell(Cell):
    @Cell.size.setter
    def size(self, value):
        if value > 500:
            raise ValueError("Слишком большой радиус")
        Cell.size.fset(self, value)  # явный вызов fset родителя

## `@cached_property`

Декоратор `cached_property` из модуля `functools` вычисляет значение свойства один раз при первом обращении и сохраняет результат в `__dict__` экземпляра. При повторных обращениях возвращается это закэшированное значение без повторного вызова функции.

Свойство `cached_property` является дескриптором не-данных и может быть реализовано только чтение.

>⚠️ Ограничение: cached_property несовместим с `__slots__`, поскольку требует `__dict__` для хранения результата.

In [13]:
from functools import cached_property
import math

class Cell:
    def __init__(self, radius):
        self._radius = radius

    @cached_property
    def volume(self):
        """Вычисляется один раз и кэшируется."""
        print("  [computing volume...]")
        return (4/3) * math.pi * self._radius ** 3

In [14]:
cell = Cell(10)

print(cell.volume)  # [computing volume...] → 4188.79...
print(cell.volume)  # 4188.79... (без повторного вычисления)

# Значение хранится в __dict__ экземпляра:
print(cell.__dict__)  # {'_radius': 10, 'volume': 4188.79...}

  [computing volume...]
4188.790204786391
4188.790204786391
{'_radius': 10, 'volume': 4188.790204786391}


Для сброса кэша достаточно удалить атрибут:

In [15]:
del cell.volume       # сбрасываем кэш
print(cell.volume)    # [computing volume...] — вычисляется заново

  [computing volume...]
4188.790204786391
